<a href="https://colab.research.google.com/github/abhinishtiwari/-House-Price-Prediction-Project/blob/main/Zone_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Adaptive Psychological Assessment System
Version: 5.2 (Minimal - Only Questions and Result)
Author: Expert Python Developer
Date: 2025-01-15

Current Date and Time (UTC - YYYY-MM-DD HH:MM:SS formatted): 2025-10-15 08:30:44
Current User's Login: abhinishtiwari
"""
import os
import sys
import json
import re
import traceback
from datetime import datetime, timezone
from collections import defaultdict, Counter

import pandas as pd


class AdaptiveBrainZoneAssessment:
    def __init__(self, excel_file="XYZ_Brain_Zones_Intake_v2.xlsx",
                 use_ai_report=True,
                 ai_provider="gemini",
                 api_key=None,
                 min_answers_trigger=4):
        self.excel_file = excel_file
        self.questions_df = None
        self.lookups_df = None
        self.responses = []
        self.core10_responses = []
        self.followup_responses = []
        self.zone_scores = defaultdict(int)
        self.zone_response_counts = defaultdict(int)
        self.extended_zone_scores = defaultdict(int)
        self.user_id = self._generate_user_id()
        self.user_name = "abhinishtiwari"
        self.current_datetime = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")
        self.current_phase = 0
        self.tie_flag = False
        self.tied_zones = []
        self.min_answers_trigger = min_answers_trigger
        self.asked_questions = set()
        self.zone_strengths = defaultdict(list)
        self.zone_challenges = defaultdict(list)

        # Tracking for first and second set of 4
        self.first4_zone_scores = defaultdict(int)
        self.second4_zone_scores = defaultdict(int)
        self.first4_leader = None
        self.second4_leader = None

        # Report generation is disabled in this build
        self.use_ai_report = False
        self.api_provider = None
        self.api_key = None
        self.llm = None

        self.load_data()

    def _generate_user_id(self):
        timestamp = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
        return f"user_{timestamp}"

    def load_data(self):
        try:
            if not os.path.exists(self.excel_file):
                raise FileNotFoundError(f"Excel file '{self.excel_file}' not found in current directory.")

            self.questions_df = pd.read_excel(self.excel_file, sheet_name="Questions")
            try:
                self.lookups_df = pd.read_excel(self.excel_file, sheet_name="Lookups")
            except Exception:
                self.lookups_df = None

            required_cols = [
                'id', 'zone', 'theme', 'question',
                'response_options', 'response_scores',
                'is_core10', 'core_order'
            ]
            missing_cols = set(required_cols) - set(self.questions_df.columns)
            if missing_cols:
                raise ValueError(f"Missing required columns: {missing_cols}")

            self.questions_df['is_core10'] = self.questions_df['is_core10'].fillna(0).astype(int)
            self.questions_df['core_order'] = self.questions_df['core_order'].fillna(999).astype(int)
            if 'priority' not in self.questions_df.columns:
                self.questions_df['priority'] = range(len(self.questions_df))

        except Exception as e:
            print(f"\nERROR: {str(e)}")
            traceback.print_exc()
            sys.exit(1)

    def clean_question_text(self, question_text):
        patterns = [
            r'^In the past \d+ weeks?,?\s*',
            r'^In the past \d+ days?,?\s*',
            r'^Over the past \d+ weeks?,?\s*',
            r'^Over the past \d+ days?,?\s*',
            r'^During the past \d+ weeks?,?\s*',
            r'^During the past \d+ days?,?\s*',
            r'^In the last \d+ weeks?,?\s*',
            r'^In the last \d+ days?,?\s*',
        ]
        cleaned = question_text
        for pattern in patterns:
            cleaned = re.sub(pattern, '', cleaned, flags=re.IGNORECASE)
        if cleaned and cleaned[0].islower():
            cleaned = cleaned[0].upper() + cleaned[1:]
        return cleaned.strip()

    def parse_response_options(self, row):
        options_str = str(row['response_options'])
        scores_str = str(row['response_scores'])
        options = [opt.strip() for opt in options_str.split(';') if opt.strip()]
        scores = [int(score.strip()) for score in scores_str.split(';') if score.strip()]
        return options, scores

    def ask_question(self, question_id, phase_name="Assessment", is_core10=False, set_num=None):
        q_row = self.questions_df[self.questions_df['id'] == question_id].iloc[0]
        options, scores = self.parse_response_options(q_row)
        cleaned_question = self.clean_question_text(q_row['question'])

        print(f"\n{cleaned_question}")

        for idx, option in enumerate(options, 1):
            print(f"  {idx}. {option}")

        while True:
            try:
                user_input = input(f"\nYour choice (1-{len(options)}): ").strip()
                choice_idx = int(user_input) - 1
                if 0 <= choice_idx < len(options):
                    selected_option = options[choice_idx]
                    selected_score = scores[choice_idx]
                    break
                else:
                    print(f"Please enter a number between 1 and {len(options)}")
            except ValueError:
                print("Invalid input. Please enter a number.")
            except KeyboardInterrupt:
                print("\n\nAssessment interrupted. Exiting...")
                sys.exit(0)

        response_data = {
            'id': question_id,
            'zone': q_row['zone'],
            'theme': q_row['theme'],
            'question': cleaned_question,
            'original_question': q_row['question'],
            'response': selected_option,
            'score': selected_score,
            'phase': phase_name,
            'is_core10': is_core10,
            'set': set_num
        }

        self.responses.append(response_data)
        self.asked_questions.add(question_id)

        if is_core10:
            self.core10_responses.append(response_data)
        else:
            self.followup_responses.append(response_data)

        self.zone_scores[q_row['zone']] += selected_score
        self.zone_response_counts[q_row['zone']] += 1
        self.extended_zone_scores[q_row['zone']] += selected_score

        # Track set-specific scores
        if set_num == 1:
            self.first4_zone_scores[q_row['zone']] += selected_score
        elif set_num == 2:
            self.second4_zone_scores[q_row['zone']] += selected_score

        if selected_score <= 1:
            self.zone_strengths[q_row['zone']].append({
                'theme': q_row['theme'],
                'question': cleaned_question,
                'response': selected_option,
                'score': selected_score
            })
        elif selected_score >= 3:
            self.zone_challenges[q_row['zone']].append({
                'theme': q_row['theme'],
                'question': cleaned_question,
                'response': selected_option,
                'score': selected_score
            })

        return response_data

    def get_core10_questions(self, start_order, count):
        core_questions = self.questions_df[self.questions_df['is_core10'] == 1].sort_values('core_order')
        selected = core_questions[
            (core_questions['core_order'] >= start_order) &
            (core_questions['core_order'] < start_order + count)
        ]
        return selected['id'].tolist()

    def get_zone_leader(self, zone_scores_dict):
        """Get the leading zone from a score dictionary"""
        if not zone_scores_dict:
            return None, 0, False, []
        max_score = max(zone_scores_dict.values())
        leaders = [zone for zone, score in zone_scores_dict.items() if score == max_score]
        if len(leaders) > 1:
            return leaders[0], max_score, True, leaders
        else:
            return leaders[0], max_score, False, leaders

    def get_recommended_followups(self, zone, count=1):
        """Get follow-up questions - only 1 per zone"""
        available = self.questions_df[
            (self.questions_df['zone'] == zone) &
            (self.questions_df['is_core10'] == 0) &
            (~self.questions_df['id'].isin(self.asked_questions))
        ].sort_values('priority')
        return available['id'].head(count).tolist()

    def ask_followup_for_zone(self, zone, set_name, count=1):
        """Ask follow-up questions for a specific zone - SILENTLY"""
        followup_ids = self.get_recommended_followups(zone, count)
        if not followup_ids:
            return

        for q_id in followup_ids:
            self.ask_question(q_id, f"Follow-up: {set_name} - {zone}", is_core10=False)

    def user_select_zone(self, tied_zones, max_score):
        """Let user select which zone they want to focus on when there's a tie"""
        print(f"\n{'='*70}")
        print("MULTIPLE ZONES TIED - Please Select Your Focus Zone")
        print(f"{'='*70}\n")

        # Display number of tied zones and their score
        num_tied = len(tied_zones)
        print(f"The following {num_tied} zones are tied at {max_score} points:\n")

        for idx, zone in enumerate(tied_zones, 1):
            print(f"  {idx}. {zone}")

        while True:
            try:
                user_input = input(f"\nYour choice (1-{len(tied_zones)}): ").strip()
                choice_idx = int(user_input) - 1
                if 0 <= choice_idx < len(tied_zones):
                    selected_zone = tied_zones[choice_idx]
                    return selected_zone
                else:
                    print(f"Please enter a number between 1 and {len(tied_zones)}")
            except ValueError:
                print("Invalid input. Please enter a number.")
            except KeyboardInterrupt:
                print("\n\nAssessment interrupted. Exiting...")
                sys.exit(0)

    def run_assessment(self):
        # ========== FIRST SET: First 4 Questions ==========
        self.current_phase = 1
        first4_questions = self.get_core10_questions(1, 4)

        for q_id in first4_questions:
            self.ask_question(q_id, "First Set", is_core10=True, set_num=1)

        # Analyze First Set - SILENTLY
        set1_leader, set1_score, set1_tie, set1_tied = self.get_zone_leader(dict(self.first4_zone_scores))
        self.first4_leader = set1_leader

        # ONLY ask follow-ups if there's a TIE
        if set1_tie:
            for zone in set1_tied:
                self.ask_followup_for_zone(zone, "First Set", count=1)

        # ========== SECOND SET: Next 4 Questions ==========
        self.current_phase = 2
        second4_questions = self.get_core10_questions(5, 4)

        for q_id in second4_questions:
            self.ask_question(q_id, "Second Set", is_core10=True, set_num=2)

        # Analyze Second Set - SILENTLY
        set2_leader, set2_score, set2_tie, set2_tied = self.get_zone_leader(dict(self.second4_zone_scores))
        self.second4_leader = set2_leader

        # ONLY ask follow-ups if there's a TIE
        if set2_tie:
            for zone in set2_tied:
                self.ask_followup_for_zone(zone, "Second Set", count=1)

        # ========== FINAL ANALYSIS: Pick Highest Zone ==========
        final_zone_totals = dict(self.extended_zone_scores)

        # Find zone with HIGHEST TOTAL SCORE
        if final_zone_totals:
            max_score = max(final_zone_totals.values())
            top_zones = [zone for zone, score in final_zone_totals.items() if score == max_score]

            if len(top_zones) == 1:
                # Clear winner
                primary_zone = top_zones[0]
                self.tie_flag = False
                self.tied_zones = []
            else:
                # Multiple zones tied - LET USER CHOOSE
                self.tie_flag = True
                self.tied_zones = top_zones
                primary_zone = self.user_select_zone(top_zones, max_score)
        else:
            primary_zone = None
            self.tie_flag = False
            self.tied_zones = []

        # ========== DISPLAY RESULT ==========
        print(f"\n{'='*70}")
        print("ASSESSMENT COMPLETE")
        print(f"{'='*70}\n")

        print("Zone Scores:")
        for zone, score in sorted(final_zone_totals.items(), key=lambda x: x[1], reverse=True):
            marker = " ← PRIMARY ZONE" if zone == primary_zone else ""
            print(f"  • {zone}: {score} points{marker}")

        print(f"\n{'='*70}")
        print(f"PRIMARY ZONE: {primary_zone}")
        print(f"{'='*70}")

        print(f"\nTotal Questions: {len(self.responses)}")
        print(f"Date & Time (UTC): {self.current_datetime}")
        print(f"User: {self.user_name}")
        print(f"{'='*70}\n")

        # ========== SAVE RESULTS ==========
        self.save_results(primary_zone=primary_zone)

        try:
            self.save_to_excel()
        except Exception:
            pass

    def get_severity_level(self, zone):
        if not zone or zone not in self.zone_scores or zone not in self.zone_response_counts:
            return "not assessed"
        avg_score = self.zone_scores[zone] / max(1, self.zone_response_counts[zone])
        if avg_score < 1.0:
            return "minimal"
        elif avg_score < 2.0:
            return "mild"
        elif avg_score < 3.0:
            return "moderate"
        else:
            return "significant"

    def save_results(self, primary_zone=None, filename_json="adaptive_result.json"):
        severity = self.get_severity_level(primary_zone) if primary_zone else "not assessed"

        output = {
            'user_id': self.user_id,
            'user_name': self.user_name,
            'assessment_date': self.current_datetime,
            'total_questions_asked': len(self.responses),
            'core_questions': len(self.core10_responses),
            'followup_questions': len(self.followup_responses),
            'responses': self.responses,
            'first4_leader': self.first4_leader,
            'second4_leader': self.second4_leader,
            'first4_scores': dict(self.first4_zone_scores),
            'second4_scores': dict(self.second4_zone_scores),
            'extended_zone_totals': dict(self.extended_zone_scores),
            'primary_zone': primary_zone,
            'primary_zone_user_selected': self.tie_flag,
            'severity_level': severity,
            'tie_detected': self.tie_flag,
            'tied_zones': self.tied_zones if self.tie_flag else [],
            'strengths': {zone: strengths for zone, strengths in self.zone_strengths.items()},
            'challenges': {zone: challenges for zone, challenges in self.zone_challenges.items()},
        }

        try:
            with open(filename_json, 'w', encoding='utf-8') as f:
                json.dump(output, f, indent=2, ensure_ascii=False)
            print(f"Results saved to {filename_json}")
        except Exception as e:
            print(f"Warning: Could not save results: {e}")

    def save_to_excel(self):
        try:
            session_df = pd.DataFrame(self.responses)
            metadata_df = pd.DataFrame([{
                'user_id': self.user_id,
                'user_name': self.user_name,
                'assessment_date': self.current_datetime,
                'total_questions': len(self.responses),
                'first4_leader': self.first4_leader,
                'second4_leader': self.second4_leader,
                'primary_zone': self.responses[-1].get('zone') if self.responses else None,
            }])

            with pd.ExcelWriter(self.excel_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                metadata_df.to_excel(writer, sheet_name='Session_Metadata', index=False)
                session_df.to_excel(writer, sheet_name='Session_Log', index=False)

        except Exception as e:
            pass


def main():
    try:
        assessment = AdaptiveBrainZoneAssessment(
            excel_file="XYZ_Brain_Zones_Intake_v2.xlsx",
            min_answers_trigger=4
        )
        assessment.run_assessment()

    except KeyboardInterrupt:
        print("\n\nAssessment interrupted. Exiting...")
        sys.exit(0)
    except Exception as e:
        print(f"\nERROR: {str(e)}")
        traceback.print_exc()
        sys.exit(1)


if __name__ == "__main__":
    main()